In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
!pip install undetected-chromedriver
import undetected_chromedriver as uc

  Using cached selenium-4.29.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.29.0-py3-none-any.whl (9.5 MB)
Using cached trio-0.29.0-py3-none-any.whl (492 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached outcome-1.3.0.pos

# Harshit's Data

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
base_url = ['https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/indian-premier-league-2009-374163/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/indian-premier-league-2009-10-418064/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/indian-premier-league-2011-466304/match-schedule-fixtures-and-results']
soups = []
for s in base_url:
    response = requests.get(s,headers=headers)
    soups.append(BeautifulSoup(response.text,'html.parser'))
    print(response.status_code)

200
200
200
200


In [5]:
allseasonmatches = []

for i,soup in enumerate(soups) :
    seasonmatch = []
    anchors = soup.find_all('a',class_='ds-inline-flex ds-items-start ds-leading-none')
    for anchor in anchors :
        link = anchor.get('href')
        link = urljoin(base_url[i],link)
        if 'live-cricket-score' in link :
            seasonmatch.append(link)
    allseasonmatches.append(seasonmatch)
allseasonmatches

[['https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match-335982/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/kings-xi-punjab-vs-chennai-super-kings-2nd-match-335983/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/delhi-daredevils-vs-rajasthan-royals-3rd-match-335984/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/kolkata-knight-riders-vs-deccan-chargers-4th-match-335986/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/mumbai-indians-vs-royal-challengers-bangalore-5th-match-335985/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/rajasthan-royals-vs-kings-xi-punjab-6th-match-335987/live-cricket-score',
  'https://www.espncricinfo.com/series/indian-premier-league-200

Columns to fill
1. Match type
2. POTM
3. Venue
4. Team1
5. Team2
6. Toss Winner
7. Toss Decision
8. Winner
9. Win margin
10. Result type(number of wickets left or run difference)
11. Target runs

## Match Details

In [6]:
def scraper(i,matchlink) :
    try :
        match_data = {
                        'Match Type': '',
                        'POTM': '',
                        'Venue': '',
                        'Team1': '',
                        'Team2': '',
                        'Toss_Winner': '',
                        'Toss_Decision': '',
                        'Winner': '',
                        'Win_Margin': '',
                        'Result_Type': '',
                        'Target_Runs': '',
                        'Season':'',
                        'Match':''
                }
    
        match_data['Season'] = f"{2008+i}"
                
        match = requests.get(matchlink,headers=headers)
        #print(match.status_code)
        soup_match = BeautifulSoup(match.text,'html.parser')
    
        driver.get(matchlink)
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
    
        elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
        potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
        teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
        tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
    
        #  Match type, POTM and Venue
        
        venue = elements.split(',')[1].strip()
        match_data['Venue'] = venue
    
        matching_word = next((word for word in ['Final','Qualifier 1','Eliminator','Qualifier 2'] if word in elements), None)
        
        if matching_word:
            match_data['Match Type'] = matching_word
            match_data['Match'] = matching_word
        else:
            match_data['Match Type'] = 'League'
            match_data['Match'] = elements.split()[:2]
        match_data['POTM'] = potmscrape
        
        # Teams
        
        match_data['Team1'] = teamscrape[0].get_text(strip=True)
        match_data['Team2'] = teamscrape[1].get_text(strip=True)
        
        # Toss Winner and Decision
        
        match_data['Toss_Winner'] = tossscrape.split(',')[0]
        if 'bat' in tossscrape :    
            match_data['Toss_Decision'] = 'bat'
        else :
            match_data['Toss_Decision'] = 'field'
        
        ## Win Details
        
        winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
        match_data['Winner'] = winscrape.split()[0]
        if 'wicket' in winscrape : 
            match_data['Result_Type'] = 'wicket'
        else :
            match_data['Result_Type'] = 'runs'
                
        match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
                
        ## Target Runs
                
        trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
        match_data['Target_Runs'] = trunscrape.split('/')[0]
    except :
        print(matchlink)
    return match_data

In [7]:

# asmd - all season match data
# smd - season match data
asmd = []
failed = [[],[],[],[],[]]

# Launch stealthy driver
driver = webdriver.Chrome()

for i,seasonmatch in enumerate(allseasonmatches):
    smd = []
    for j,matchlink in enumerate(seasonmatch):
        try :
            match_data = {
                    'Match Type': '',
                    'POTM': '',
                    'Venue': '',
                    'Team1': '',
                    'Team2': '',
                    'Toss_Winner': '',
                    'Toss_Decision': '',
                    'Winner': '',
                    'Win_Margin': '',
                    'Result_Type': '',
                    'Target_Runs': '',
                    'Season':'',
                    'Match':''
            }

            match_data['Season'] = f"{2008+i}"
            
            match = requests.get(matchlink,headers=headers)
            #print(match.status_code)
            soup_match = BeautifulSoup(match.text,'html.parser')

            driver.get(matchlink)
    
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
    
            elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
            potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
            teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
            tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
    
            #  Match type, POTM and Venue
            
            venue = elements.split(',')[1].strip()
            match_data['Venue'] = venue
    
            matching_word = next((word for word in ['Final','Qualifier 1','Eliminator','Qualifier 2'] if word in elements), None)
            
            if matching_word:
                match_data['Match Type'] = matching_word
                match_data['Match'] = matching_word
            else:
                match_data['Match Type'] = 'League'
                match_data['Match'] = elements.split()[:2]
            match_data['POTM'] = potmscrape
            
            # Teams
            
            match_data['Team1'] = teamscrape[0].get_text(strip=True)
            match_data['Team2'] = teamscrape[1].get_text(strip=True)
            
            # Toss Winner and Decision
            
            match_data['Toss_Winner'] = tossscrape.split(',')[0]
            if 'bat' in tossscrape :    
                match_data['Toss_Decision'] = 'bat'
            else :
                match_data['Toss_Decision'] = 'field'
            
            ## Win Details
            
            winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
            match_data['Winner'] = winscrape.split()[0]
            if 'wicket' in winscrape : 
                match_data['Result_Type'] = 'wicket'
            else :
                match_data['Result_Type'] = 'runs'
            
            match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
            
            ## Target Runs
            
            trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
            match_data['Target_Runs'] = trunscrape.split('/')[0]
        except :
            failed[i].append(matchlink)
        smd.append(match_data)
    asmd.append(smd)
driver.quit()
print(failed)

[['https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/delhi-daredevils-vs-kolkata-knight-riders-47th-match-336030/live-cricket-score'], ['https://www.espncricinfo.com/series/indian-premier-league-2009-374163/mumbai-indians-vs-rajasthan-royals-7th-match-392187/live-cricket-score', 'https://www.espncricinfo.com/series/indian-premier-league-2009-374163/kolkata-knight-riders-vs-rajasthan-royals-10th-match-392190/live-cricket-score', 'https://www.espncricinfo.com/series/indian-premier-league-2009-374163/chennai-super-kings-vs-kolkata-knight-riders-13th-match-392193/live-cricket-score'], ['https://www.espncricinfo.com/series/indian-premier-league-2009-10-418064/chennai-super-kings-vs-kings-xi-punjab-16th-match-419121/live-cricket-score'], ['https://www.espncricinfo.com/series/indian-premier-league-2011-466304/royal-challengers-bangalore-vs-rajasthan-royals-20th-match-501217/live-cricket-score', 'https://www.espncricinfo.com/series/indian-premier-league-2011-466304/delhi

In [9]:
frames = []
for smd in asmd :
    df = pd.DataFrame(smd)
    frames.append(df)
result = pd.concat(frames).reset_index()
result.to_csv(r'E:\PYTHON\Udemy\Web_Scraping\ipl_da_08_11.csv')
result

,index,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,0,League,Brendon McCullum,Bengaluru,KKR,RCB,Royal Challengers Bangalore,field,KKR,140,runs,222,2008,"[1st, match]"
1,1,League,Michael Hussey,Mohali,CSK,KXIP,Chennai Super Kings,bat,CSK,33,runs,240,2008,"[2nd, match]"
2,2,League,Farveez Maharoof,Delhi,RR,DC,Rajasthan Royals,bat,Daredevils,9,wicket,129,2008,"[3rd, match]"
3,3,League,David Hussey,Eden Gardens,DCH,KKR,Deccan Chargers,bat,KKR,5,wicket,110,2008,"[4th, match]"
4,4,League,Mark Boucher,Wankhede,MI,RCB,Mumbai Indians,bat,RCB,5,wicket,165,2008,"[5th, match]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,69,League,James Franklin,Eden Gardens,KKR,MI,Mumbai Indians,field,MI,5,wicket,175,2011,"[70th, match]"
248,70,Final,Suresh Raina,Wankhede,RCB,CSK,Chennai Super Kings,field,CSK,6,wicket,175,2011,Final
249,71,Final,Munaf Patel,Wankhede,KKR,MI,Mumbai Indians,field,MI,4,wicket,147,2011,Final
250,72,Final,Chris Gayle,Chennai,RCB,MI,Mumbai Indians,field,RCB,43,runs,185,2011,Final


In [10]:
driver = webdriver.Chrome()
miss = []
for i,season in enumerate(failed) :
    for matchlink in season :
        if matchlink :
            miss.append(scraper(i,matchlink))
driver.quit()
missdf = pd.DataFrame(miss)
missdf

https://www.espncricinfo.com/series/indian-premier-league-2007-08-313494/delhi-daredevils-vs-kolkata-knight-riders-47th-match-336030/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2009-374163/mumbai-indians-vs-rajasthan-royals-7th-match-392187/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2009-374163/kolkata-knight-riders-vs-rajasthan-royals-10th-match-392190/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2009-374163/chennai-super-kings-vs-kolkata-knight-riders-13th-match-392193/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2009-10-418064/chennai-super-kings-vs-kings-xi-punjab-16th-match-419121/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2011-466304/royal-challengers-bangalore-vs-rajasthan-royals-20th-match-501217/live-cricket-score
https://www.espncricinfo.com/series/indian-premier-league-2011-466304/delhi-daredevils-vs-pune-warrior

,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,,,,,,,,,,,,2008,
1,,,,,,,,,,,,2009,
2,League,Yusuf Pathan,Cape Town,RR,KKR,Kolkata Knight Riders,field,Match,,runs,,2009,"[10th, match]"
3,,,,,,,,,,,,2009,
4,League,Rusty Theron,Chennai,KXIP,CSK,Chennai Super Kings,field,Match,,runs,,2010,"[16th, match]"
5,,,,,,,,,,,,2011,
6,,,,,,,,,,,,2011,


In [11]:
empty_rows = result[result['Match Type'].isna()].index
for idx, miss_row in zip(empty_rows, missdf.iterrows()):
    result.loc[idx] = miss_row[1]
result.to_csv(r'E:\PYTHON\Udemy\Web_Scraping\ipl_da_08_11.csv')

In [12]:
result[result['Match Type']=='']['Match Type'].count()

np.int64(5)